## Merge Fundamental + Momentum + Tecnico


In [2]:
import nasdaqdatalink as ndl
import pandas as pd
from ta import add_all_ta_features
import pandas as pd
import zipfile
import json

In [81]:
#configurations
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


## Importamos datasets a mergear

In [8]:
rebalance_returns = pd.read_csv('../data/rebalance_returns.csv', parse_dates=['rebalance_date'])
tecnico = pd.read_csv('../data/tecnico.csv',          parse_dates=['rebalance_date'])
fundamental = pd.read_csv("../data/fundamental.csv",       parse_dates=["date"])
sp500 = pd.read_csv("../data/sp500.csv", parse_dates=["date"])

In [4]:
fundamental.head(10)

,None,ticker,datekey,date,revenue,eps,ebitda,netinc,roe,evebitda,...,pb,pe,marketcap,ebit,currentratio,capex,ncfo,equity,de,year
0,0,ZTS,2025-02-13,2024-12-31,9.256000e+09,5.47,3.845000e+09,2.486000e+09,0.497,20.445,...,15.483,29.708,7.385432e+10,3.348000e+09,1.754,-654000000.0,2.953000e+09,4.770000e+09,1.985,2024
1,1,ZTS,2024-02-13,2023-12-31,8.544000e+09,5.08,3.670000e+09,2.344000e+09,0.488,24.177,...,16.813,35.842,8.401404e+10,3.179000e+09,3.358,-728000000.0,2.353000e+09,4.997000e+09,1.860,2023
2,2,ZTS,2023-02-14,2022-12-31,8.080000e+09,4.51,3.345000e+09,2.114000e+09,0.462,25.161,...,18.083,37.680,7.965618e+10,2.880000e+09,2.370,-585000000.0,1.912000e+09,4.405000e+09,2.389,2022
3,3,ZTS,2022-02-15,2021-12-31,7.776000e+09,4.29,3.163000e+09,2.037000e+09,0.461,30.539,...,20.545,45.821,9.333690e+10,2.715000e+09,3.856,-475000000.0,2.213000e+09,4.543000e+09,2.059,2021
4,4,ZTS,2021-02-16,2020-12-31,6.675000e+09,3.44,2.670000e+09,1.638000e+09,0.500,31.007,...,20.968,48.248,7.902967e+10,2.229000e+09,3.047,-432000000.0,2.126000e+09,3.769000e+09,2.610,2020
5,5,ZTS,2020-02-13,2019-12-31,6.260000e+09,3.14,2.436000e+09,1.500000e+09,0.593,30.178,...,25.420,45.891,6.883693e+10,2.024000e+09,2.629,-460000000.0,1.795000e+09,2.708000e+09,3.263,2019
6,6,ZTS,2019-02-14,2018-12-31,5.825000e+09,2.96,2.208000e+09,1.428000e+09,0.694,22.419,...,20.435,31.268,4.465027e+10,1.900000e+09,3.597,-338000000.0,1.790000e+09,2.185000e+09,3.932,2018
7,7,ZTS,2018-02-15,2017-12-31,5.307000e+09,1.76,1.944000e+09,8.640000e+08,0.488,21.338,...,21.521,44.088,3.809242e+10,1.702000e+09,3.855,-224000000.0,1.346000e+09,1.770000e+09,3.842,2017
8,8,ZTS,2017-02-16,2016-12-31,4.888000e+09,1.66,1.636000e+09,8.210000e+08,0.587,18.158,...,17.462,31.627,2.596586e+10,1.396000e+09,3.035,-216000000.0,7.130000e+08,1.487000e+09,4.136,2016
9,9,ZTS,2016-02-24,2015-12-31,4.765000e+09,0.68,8.680000e+08,3.390000e+08,0.285,28.547,...,19.723,62.137,2.106448e+10,6.690000e+08,2.150,-224000000.0,6.640000e+08,1.068000e+09,6.388,2015


In [11]:

# 1. Ordenar bien el fundamental antes de pct_change
fundamental = (
    fundamental
      .sort_values(['ticker', 'datekey'])  # MUY importante: por fecha de publicación
      .reset_index(drop=True)
)

# 2. Calcular el crecimiento YoY (% Change respecto al año anterior) para variables fundamentales
for col in ['revenue', 'ebitda', 'ebit', 'eps', 'equity', 'capex', 'pe', 'pb', 'fcfps', 'ncfo']:
    fundamental[f'{col}_yoy'] = fundamental.groupby('ticker')[col].pct_change()


C:\Users\Ignacio\AppData\Local\Temp\ipykernel_9484\842037972.py:10: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  fundamental[f'{col}_yoy'] = fundamental.groupby('ticker')[col].pct_change()
C:\Users\Ignacio\AppData\Local\Temp\ipykernel_9484\842037972.py:10: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  fundamental[f'{col}_yoy'] = fundamental.groupby('ticker')[col].pct_change()
C:\Users\Ignacio\AppData\Local\Temp\ipykernel_9484\842037972.py:10: FutureWarning: The default fill_method='ffill' in SeriesGroupBy.pct_change is deprecated and will be removed in a future version. Either

In [12]:
fundamental.head()

,None,ticker,datekey,date,revenue,eps,ebitda,netinc,roe,evebitda,...,revenue_yoy,ebitda_yoy,ebit_yoy,eps_yoy,equity_yoy,capex_yoy,pe_yoy,pb_yoy,fcfps_yoy,ncfo_yoy
0,20726,A,2000-01-25,1999-12-31,8.331000e+09,1.35,1.262000e+09,5.120000e+08,NaN,24.244,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,20725,A,2001-01-17,2000-12-31,1.077300e+10,1.68,1.659000e+09,7.570000e+08,0.157,16.505,...,0.293122,0.314580,0.479034,0.244444,0.556771,0.950000,-0.375130,-0.406544,0.139098,0.817787
2,20724,A,2002-01-22,2001-12-31,8.396000e+09,0.38,8.370000e+08,1.740000e+08,0.031,13.656,...,-0.220644,-0.495479,-0.911512,-0.773810,0.074834,-0.153846,0.938325,-0.585584,-6.102310,-1.136038
3,20723,A,2002-02-01,2001-12-31,8.396000e+09,0.38,8.370000e+08,1.740000e+08,0.031,15.013,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090206,0.090337,0.000000,0.000000
4,20722,A,2002-12-20,2002-12-31,6.010000e+09,-2.22,-8.220000e+08,-1.032000e+09,-0.205,-9.383,...,-0.284183,-1.982079,-16.116505,-6.842105,-0.182364,-0.493266,-1.103234,-0.251031,0.232212,4.131579


In [13]:
snapshot_0331 = (
    sp500
      .sort_values(['ticker','date'])
      .reset_index(drop=True)
)


In [14]:
# extrae lista de (ticker, rebalance_date) únicos
tecnico = (
    tecnico
      .drop_duplicates()
      .sort_values(['ticker','rebalance_date'])
      .reset_index(drop=True)
)

snapshot_0331['date'] = pd.to_datetime(snapshot_0331['date'])
tecnico['rebalance_date'] = pd.to_datetime(tecnico['rebalance_date'])

# Ordena ambos DataFrames por 'ticker' y la columna de fecha
snapshot_0331 = snapshot_0331.sort_values(['date']).reset_index(drop=True)
tecnico = tecnico.sort_values(['rebalance_date']).reset_index(drop=True)

# para cada snapshot_0331.calendar_date toma el rebalance_date ≤ calendar_date
constituents_and_tecnico= pd.merge_asof(
    snapshot_0331,
    tecnico,
    left_on   = 'date',
    right_on  = 'rebalance_date',
    by        = 'ticker',
    direction = 'backward'
).rename(columns={'rebalance_date':'trade_date'})


In [15]:
fundamental = (
    fundamental
      .rename(columns={'date':'financial_year_end_date','datekey':'report_date'})
      .sort_values(['ticker','report_date'])
      .reset_index(drop=True)
)
fundamental['report_date'] = pd.to_datetime(fundamental['report_date'])
#fundamental['financial_year_end_date'] = pd.to_datetime(fundamental['financial_year_end_date'])


In [16]:
constituents_tecnico_fundamental = pd.merge_asof(
    constituents_and_tecnico.sort_values(['date']),
    fundamental.sort_values(['report_date']),
    left_on   = 'date',
    right_on  = 'report_date',
    by        = 'ticker',
    direction = 'backward'
)


In [17]:
merged = pd.merge(
    constituents_tecnico_fundamental.sort_values(['ticker', 'date']),
    rebalance_returns.sort_values(['ticker', 'rebalance_date']),
    left_on   = ['ticker', 'date'],
    right_on  = ['ticker', 'rebalance_date'],
    how='left'  # o 'inner'
)


### Lehmnan Brothers quiebra en 2008 y sale del indice

In [18]:
merged[merged['ticker'] == 'LEHMQ']

,date,ticker,year_x,trade_date,close_reb_adj,vol_1y,max_1y,min_1y,sma_1y,sma_2y,...,equity_yoy,capex_yoy,pe_yoy,pb_yoy,fcfps_yoy,ncfo_yoy,rebalance_date,closeadj,ret_6m,ret_12m
7515,1998-03-31,LEHMQ,1998.0,1998-03-31,17.443,1.884412e+08,17.443,11.532,14.137410,14.137410,...,NaN,NaN,NaN,NaN,NaN,NaN,1998-03-31,17.443,NaN,NaN
7516,1999-03-31,LEHMQ,1999.0,1999-03-31,13.920,1.263827e+09,19.802,5.766,13.279135,13.446403,...,NaN,NaN,NaN,NaN,NaN,NaN,1999-03-31,13.920,1.105900,-0.201972
7517,2000-03-31,LEHMQ,2000.0,2000-03-31,22.598,9.325708e+08,24.563,11.241,15.366945,14.318505,...,0.160724,-0.386555,-0.190012,0.115483,-1.606756,-1.620676,2000-03-31,22.598,0.661740,0.623420
7518,2001-03-31,LEHMQ,2001.0,2001-03-30,29.214,1.437350e+09,39.940,16.890,28.075321,21.691683,...,0.238421,2.931507,0.294974,0.692485,-3.350638,-3.359545,2001-03-31,29.214,-0.151274,0.292769
7519,2002-03-31,LEHMQ,2002.0,2002-03-28,30.277,1.211465e+09,38.323,21.731,31.123540,29.563686,...,0.087135,3.672474,0.127578,-0.282737,-1.375150,-1.453336,2002-03-31,30.277,0.143046,0.036387
7520,2003-03-31,LEHMQ,2003.0,2003-03-31,27.416,1.329330e+09,31.158,20.071,26.857822,28.967948,...,0.057099,-0.624161,0.285070,-0.051169,3.385367,2.662075,2003-03-31,27.416,0.186018,-0.094494
7521,2004-03-31,LEHMQ,2004.0,2004-03-31,39.823,1.317918e+09,43.009,28.461,34.583921,30.710149,...,0.473272,-0.105159,-0.018891,0.211718,-0.923463,-0.895867,2004-03-31,39.823,0.205844,0.452546
7522,2005-03-31,LEHMQ,2005.0,2005-03-31,45.649,1.134938e+09,46.638,32.577,39.058124,36.816575,...,0.132534,-0.110865,-0.223903,-0.045055,-6.416931,-5.283471,2005-03-31,45.649,0.186490,0.146297
7523,2006-03-31,LEHMQ,2006.0,2006-03-31,70.602,1.171674e+09,72.531,42.444,56.286870,47.706681,...,0.125603,0.019950,0.034733,0.276755,-0.286872,-0.313657,2006-03-31,70.602,0.245163,0.546628
7524,2007-03-31,LEHMQ,2007.0,2007-03-30,68.991,1.107034e+09,84.308,58.160,71.002135,63.565796,...,0.142730,0.432763,-0.053177,0.023434,3.811533,3.857906,2007-03-31,68.991,-0.047493,-0.022818


#### Seguimos teniendo 500 empresas aprox para cada ventana... excelente!

In [21]:
merged[merged['date'] == '1999-03-31']['ticker'].nunique()

500

In [22]:
merged.to_csv('../data/merged.csv', index=False)